In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
pyspark.__version__

'3.5.5'

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

25/03/06 01:13:25 WARN Utils: Your hostname, Varuns-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.29.135 instead (on interface en0)
25/03/06 01:13:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/06 01:13:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet

--2025-03-05 02:57:26--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 2600:9000:21f8:2800:b:20a5:b140:21, 2600:9000:21f8:7800:b:20a5:b140:21, 2600:9000:21f8:8c00:b:20a5:b140:21, ...
connected. to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|2600:9000:21f8:2800:b:20a5:b140:21|:443... 
HTTP request sent, awaiting response... 200 OK
Length: 64346071 (61M) [binary/octet-stream]
Saving to: ‘yellow_tripdata_2024-10.parquet’

yellow_tripdata_202 100%[===================>]  61.36M  11.7MB/s    in 6.2s    

2025-03-05 02:57:33 (9.95 MB/s) - ‘yellow_tripdata_2024-10.parquet’ saved [64346071/64346071]



In [5]:
df = spark.read.parquet('yellow_tripdata_2024-10.parquet')

In [6]:
df.count()

3833771

In [7]:
df = df.repartition(4)

In [9]:
df.write.parquet('data/homework/', mode='overwrite')

In [12]:
df.createOrReplaceTempView('data')

In [25]:
df.show()

[Stage 31:====================================>                     (5 + 3) / 8]

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2024-10-07 16:40:43|  2024-10-07 18:10:56|              1|         14.8|        99|                 N|         127|         225|           1|       47.5|  0.0|    0.5|       0.

In [11]:
df.createOrReplaceTempView("data")

In [12]:
df_result = spark.sql("""
SELECT COUNT(1) as trip_count
FROM data
WHERE tpep_pickup_datetime >= "2024-10-15 00:00:00" and tpep_dropoff_datetime <= "2024-10-15 23:59:59"
""")

In [13]:
df_result.show()

+----------+
|trip_count|
+----------+
|    127993|
+----------+



In [22]:
df_new_result = spark.sql("""
SELECT 
    MAX(EXTRACT(HOUR FROM (tpep_dropoff_datetime - tpep_pickup_datetime))) AS longest_trip_hours
FROM data;
""")

In [25]:
!wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv

--2025-03-06 01:32:57--  https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 2600:9000:21f8:fc00:b:20a5:b140:21, 2600:9000:21f8:400:b:20a5:b140:21, 2600:9000:21f8:c000:b:20a5:b140:21, ...
connected. to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|2600:9000:21f8:fc00:b:20a5:b140:21|:443... 
HTTP request sent, awaiting response... 200 OK
Length: 12331 (12K) [text/csv]
Saving to: ‘taxi_zone_lookup.csv.1’

taxi_zone_lookup.cs 100%[===================>]  12.04K  --.-KB/s    in 0.02s   

2025-03-06 01:32:58 (695 KB/s) - ‘taxi_zone_lookup.csv.1’ saved [12331/12331]



In [26]:
trips_df = spark.read.parquet("yellow_tripdata_2024-10.parquet")

In [27]:
zones_df = spark.read.csv("taxi_zone_lookup.csv", header=True, inferSchema=True)

In [28]:
trips_df.createOrReplaceTempView("trips")
zones_df.createOrReplaceTempView("zones")

In [29]:
result = spark.sql( """
SELECT z.Zone, COUNT(t.PULocationID) as pickup_count
FROM trips t
JOIN zones z ON t.PULocationID = z.LocationID
GROUP BY z.Zone
ORDER BY pickup_count ASC
LIMIT 1
""")

In [30]:
result.show()

[Stage 30:=============================>                            (4 + 4) / 8]

+--------------------+------------+
|                Zone|pickup_count|
+--------------------+------------+
|Governor's Island...|           1|
+--------------------+------------+

